## Solve the linear system
We want to solve the linear system $$Ax=b$$ and we split the matrix into first unknown matrices $M$ and $N$
as follows 
$$A=M-N$$
where we assume that $M$ is invertible. We can then write our linear system as follows
\begin{align}
Ax&=b\\
\Leftrightarrow&\quad =b\\
\Leftrightarrow&\quad  Mx-Nx=b\\
\Leftrightarrow&\quad  Mx=b+Nx\\
\Leftrightarrow&\quad  x=M^{-1}b+M^{-1}Nx\\
\Leftrightarrow&\quad  x=M^{-1}b+M^{-1}(M-A)x\\
\Leftrightarrow&\quad  x=M^{-1}b+(I-M^{-1}A)x\\
\Leftrightarrow&\quad  x=x+M^{-1}(b-Ax)\\
\end{align}
so far it seems nothing is gained but let's turn this into an iteration
$$
x^{(k+1)}=x^{(k)}+M^{-1}(b-Ax^{(k)}).
$$
This method does work quite well under certain circumstances. We now want to create an actual example and for that choose 
$$
M=D
$$
with $D$ the diagonal of $A$. This is the so-called Jacobi iteration. Please implement a function that given a right hand side and a matrix implements this method. You can use the norm of the residual as a measure of convergence, i.e.,
$$
r^{(k)}=b-Ax^{(k)}.
$$
You can start with the zero vector for $x^{(0)}$.


In [1]:
import numpy as np
import scipy
from scipy import linalg as la
from scipy import optimize
import scipy.sparse as spsp

In [2]:
# function should return solution x and 
# number of iteration, such it can be called as
# (x,it)=jacobi(A,b,maxiter,tol,verbose)
# if verbose==0 the function should not print out messages
def jacobi(A,b,maxiter,tol,verbose):
    xOld=np.zeros(len(b))
    D = np.diag(np.diag(A))
    LU = A - D
    DInv = np.diag(1 / np.diag(D))
    for i in range(maxiter):
        
        xNew = np.dot(DInv, b - np.dot(LU, xOld))
        if np.linalg.norm(xNew - xOld) < tol:
            if verbose==1:
                print(f'number of iteration:{i}')
            return (xNew ,i)
        xOld = xNew    

Now check out if our iteration converges, at first with random matrx and random right hand side

In [3]:
n = 50
A = np.random.rand(n,n)
b = np.random.rand(n,1)

In [4]:
(x,it)=jacobi(A,b,10,1e-5,1)
print(x)

TypeError: cannot unpack non-iterable NoneType object

It seems not to converge, lets try another matrix, for example we make A diagonal dominant by replacing 
$A_{ii}$ with the sum of row $i$.



In [9]:
A2=A.copy()
n1=np.ones((n,1))
rs=A2@n1
#print(n1)
#print(rs)

for i in range (n):
    A2[i,i]=rs[i]
#print(A2)   
(x,it)=jacobi(A2,b,1000,1e-5,1)
print(x)

number of iteration:561
[[0.01980736 0.01980736 0.01980736 ... 0.01980736 0.01980736 0.01980736]
 [0.00326201 0.00326201 0.00326201 ... 0.00326201 0.00326201 0.00326201]
 [0.02849689 0.02849689 0.02849689 ... 0.02849689 0.02849689 0.02849689]
 ...
 [0.01135086 0.01135086 0.01135086 ... 0.01135086 0.01135086 0.01135086]
 [0.02267074 0.02267074 0.02267074 ... 0.02267074 0.02267074 0.02267074]
 [0.00785194 0.00785194 0.00785194 ... 0.00785194 0.00785194 0.00785194]]


The Jacobi iteration computes the vector 
$ x^{(k+1)} $ from values of vector $ x^{(k)} $. 
Now the idea comes up to use for the computation of the component $i$ of $x^{(k+1)}$ the already computed components $x^{(k+1)}_1 \cdots x^{(k+1)}_{i-1}$ from the new vector $x^{(k+1)}$  instead the components of the old vector $x^{(k)}$.

This leads to the Gauss-Seidel iteration,
instead of choosing
$$ M=D $$ with $D$ the diagonal of $A$ in the Jacobi iteration now choose
$$ M=(D+L) $$ where $L$ is the lower triangle of $A$.

Please implement again a function that given a right hand side and a matrix implements this method. 
You can use the norm of the residual as a measure of convergence, i.e.,
$$
r^{(k)}=b-Ax^{(k)}.
$$
You can start with the zero vector for $x^{(0)}$.


In [10]:
# function should return solution x and 
# number of iteration, such it can be called as
# (x,it)=gauss_seidel(A,b,maxiter,tol,verbose)
# if verbose==0 the function should not print out messages
def gauss_seidel(A,b,maxiter,tol,verbose):
    xOld=np.zeros(len(b))
    M = np.tril(A)
    MI=np.linalg.inv(M)
    U = A - M
    MIb=MI@b
    MIU=MI@U
    for i in range(maxiter):
        xNew = MIb-MIU@xOld
        if np.linalg.norm(xNew - xOld) < tol:
            if verbose==1:
                print(f'number of iteration:{i}')
            return (xNew ,i)
        xOld = xNew    

    return (xold,k)


In [11]:
(x,it)=gauss_seidel(A2,b,1000,1e-5,verbose=1)
print(x)
print(it)

number of iteration:8
[[0.01980744 0.01980744 0.01980744 ... 0.01980744 0.01980744 0.01980744]
 [0.0032621  0.0032621  0.0032621  ... 0.0032621  0.0032621  0.0032621 ]
 [0.02849698 0.02849698 0.02849698 ... 0.02849698 0.02849698 0.02849698]
 ...
 [0.01135096 0.01135096 0.01135096 ... 0.01135096 0.01135096 0.01135096]
 [0.02267083 0.02267083 0.02267083 ... 0.02267083 0.02267083 0.02267083]
 [0.00785204 0.00785204 0.00785204 ... 0.00785204 0.00785204 0.00785204]]
8


Now we want to examine the convergence of Jacobi and Gauss-Seidel iteration for different system sizes.
Create a plot of iteration numbers for system sizes $n=5,10,15,20,25,30,40,50$. Use a matrix $A$ and vector $b$
of your choice but ensure that this matrix will converge at all.

In [ ]:
# TODO


# get the plot lib 
import matplotlib.pyplot as plt


# TODO

#plt.show
    
    